In [113]:
import numpy as np
from pydrake.geometry import StartMeshcat, MeshcatVisualizer
from pydrake.multibody.plant import (AddMultibodyPlantSceneGraph, 
                                     MultibodyPlant)

from pydrake.multibody.parsing import Parser
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.analysis import Simulator
from pydrake.systems.controllers import InverseDynamicsController
from pydrake.systems.controllers import InverseDynamics
from pydrake.common import FindResourceOrThrow

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7004


In [41]:
# generic system
robot_path_fileDescription = FindResourceOrThrow("drake/manipulation/models/iiwa_description/sdf/iiwa14_no_collision.sdf")

plant = MultibodyPlant(time_step=1e-4)
#load the robot inside the multibody_plant
# multibody can contains several 
Parser(plant).AddModelFromFile(robot_path_fileDescription)

plant.WeldFrames(plant.world_frame(), plant.GetFrameByName('iiwa_link_0'))
plant.Finalize()

In [10]:
.GetFrameByName('iiwa_link_0'))

pydrake.multibody.tree.BodyFrame_[float]

In [42]:
# generic drake framework: system ---> function f(context)
# context ---> state

context = plant.CreateDefaultContext()
print(context.get_discrete_state_vector())
# all 0 --> position and velocity for each var
# aka q, q_dot --> 7dof times 2

# set position of joint: 
print('random_config')
plant.SetPositions(context, np.random.randn(7))
print(context.get_discrete_state_vector())

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
random_config
[-0.5191832628275186, -1.730375684753413, -0.24739189137669243, -0.42090212713912284, -0.16624825195448642, -0.42427593384013496, -0.11628720021119471, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [43]:
# input value for actuator: 
plant.get_actuation_input_port().FixValue(context, np.zeros(7))

In [44]:
simulator = Simulator(plant, context)
simulator.AdvanceTo(5.0)
print(context.get_discrete_state_vector())

# ps: notice that kuka now has also some velocity at this point
# see if exist convergence increasing simulation time

# create second context
context = plant.CreateDefaultContext()
plant.SetPositions(context, np.random.randn(7))
plant.get_actuation_input_port().FixValue(context, np.zeros(7))
simulator = Simulator(plant, context)
simulator.AdvanceTo(50.0)
print("look for convergence (q_dot=0) ---> 30 sec of simulation")
print(context.get_discrete_state_vector())

[-0.41354386093118506, -2.095336220256333, -2.9670833163318355, -0.32485017067098965, -1.4284979375027322, -1.1120102354358716, 2.960670058968681, 0.3342019796410753, 0.004247256040654124, 0.0008816438801614312, 3.122700312724296, -0.4363841592025109, 5.814237375298399, -2.089584418143337]
look for convergence (q_dot=0) ---> 30 sec of simulation
[1.0107507783745155, 2.09556966507684, 2.9671575580210523, 0.6012664220494293, -2.7743327122226265, -1.316653199330957, -0.015713574124220534, -0.12947732270186757, 0.002241860383486116, 0.0015961033794064106, 0.8438416405930916, -7.571533915418584, 7.2038563750007345, -2.729442972094217]


In [45]:
context = plant.CreateDefaultContext()
plant.SetPositions(context, np.random.randn(7))
plant.get_actuation_input_port().FixValue(context, np.zeros(7))
simulator = Simulator(plant, context)
simulator.AdvanceTo(50.0)
print("look for convergence (q_dot=0) ---> 30 sec of simulation")
print(context.get_discrete_state_vector())

look for convergence (q_dot=0) ---> 30 sec of simulation
[-0.272982640916335, 2.095919789347157, 2.967360427336056, 1.1583029993270493, 0.6286653599194674, -0.3895968624270203, -2.29363825482834, -0.058800813252463745, 4.614775556230355e-05, -1.874022126271968e-05, -0.7083843812898154, 0.6895225948808736, 0.8737295527765317, -3.0508454910593357]


In [245]:
meshcat.Delete()
meshcat.DeleteAddedControls()
builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
Parser(plant, scene_graph).AddModelFromFile(robot_path_fileDescription)
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName('iiwa_link_0'))
plant.Finalize()




In [246]:
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
diagram = builder.Build()

In [247]:
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(context)
plant.SetPositions(plant_context, np.random.randn(7))
plant.get_actuation_input_port().FixValue(plant_context, np.zeros(7))
diagram.ForcedPublish(context)


In [248]:
simulator = Simulator(diagram, context)
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(5.0)

<h3> Controller part </h3>

In [209]:
##############################################
##############################################
##############################################

meshcat.Delete()
meshcat.DeleteAddedControls()

#CONTROLLER MOUNT : 
 
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
# Set plant 
robot_model = Parser(plant, scene_graph).AddAllModelsFromFile(robot_path_fileDescription)[0]
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName('iiwa_link_0'))
plant.Finalize()

visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)


'''
TEST GRAVITY COMPENSATION 
'''
mode = InverseDynamics.InverseDynamicsMode(1)
controller= builder.AddSystem(InverseDynamics(plant, mode=mode))
controller.set_name('controller')
builder.Connect(plant.get_state_output_port(robot_model), controller.get_input_port())
builder.Connect(controller.get_output_port(), plant.get_actuation_input_port())
'''
PID CONTROLLER 
kp = [100] * plant.num_positions()
ki = [1] * plant.num_positions()
kd = [20] * plant.num_positions()
controller = builder.AddSystem(InverseDynamicsController(plant, kp, ki, kd, False))
controller.set_name("controller")
builder.Connect(plant.get_state_output_port(robot_model),controller.get_input_port_estimated_state())
builder.Connect(controller.get_output_port_control(), plant.get_actuation_input_port())
'''
diagram = builder.Build()

In [197]:
port = controller.GetInputPort('u0')
context = diagram.CreateDefaultContext()
context_controller = controller.GetMyMutableContextFromRoot(context)
print(context_controller)

::_::controller Context
------------------------
Time: 0



In [244]:

context = diagram.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(context)
q0 = np.random.rand(7)
q0_dot = np.zeros(7)
x0 = np.hstack((q0, q0_dot))

plant.SetPositions(plant_context, q0)
controller.GetInputPort('u0').FixValue( # gravity comp
#controller.GetInputPort('desired_state').FixValue( # pid
    controller.GetMyMutableContextFromRoot(context), x0
)



sim = Simulator(diagram, context)
sim.set_target_realtime_rate(1.0)
sim.AdvanceTo(5.0)